<a href="https://colab.research.google.com/github/zilioalberto/N3_Ciencia_Dados/blob/main/02_modelagem_N3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# N3 — 02_modelagem_N3 (Treino, comparação e seleção do melhor modelo)

Este notebook executa a **Parte 3** da avaliação N3:
- treina **3 modelos**;
- avalia com **3 métricas** (MAE, RMSE, R²);
- gera **tabela comparativa**;
- seleciona o melhor modelo e salva em `modelo_final.pkl` (com backup automático).

**Dados lidos diretamente do GitHub (RAW):**
- `base_modelagem.csv`
- `etl_report.json`


In [21]:
from __future__ import annotations

import json
import datetime
from pathlib import Path
from urllib.request import urlopen

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

# ----------------------------
# URLs RAW (leitura direto do GitHub)
# ----------------------------
CSV_URL = "https://raw.githubusercontent.com/zilioalberto/N3_Ciencia_Dados/main/data/dataset_processado_N3/base_modelagem.csv"
REPORT_URL = "https://raw.githubusercontent.com/zilioalberto/N3_Ciencia_Dados/main/data/dataset_processado_N3/etl_report.json"

def read_json_from_url(url: str) -> dict:
    with urlopen(url) as resp:
        content = resp.read().decode("utf-8")
    return json.loads(content)

def find_project_root(start: Path | None = None) -> Path:
    """Acha a raiz do projeto local (para salvar resultados e modelo)."""
    start = (start or Path.cwd()).resolve()
    for p in [start, *start.parents]:
        if (p / "requirements.txt").exists() or (p / "README.md").exists() or (p / "data").exists():
            return p
    return start

ROOT = find_project_root()
OUT_DIR = ROOT / "data" / "dataset_processado_N3"  # saída local (para commitar depois)
OUT_DIR.mkdir(parents=True, exist_ok=True)

ROOT, OUT_DIR


(PosixPath('/content'), PosixPath('/content/data/dataset_processado_N3'))

In [22]:
# 1) Carregar a base modelagem direto do GitHub
df = pd.read_csv(CSV_URL)

# 2) Carregar o etl_report.json direto do GitHub (se existir)
report = {}
try:
    report = read_json_from_url(REPORT_URL)
except Exception as e:
    print("Aviso: não foi possível ler etl_report.json do GitHub. Motivo:", repr(e))
    report = {}

print("Shape base_modelagem:", df.shape)
display(df.head(3))
report


Shape base_modelagem: (7780, 16)


,area,iptu,taxa_condominial,num_quartos,num_banheiros,num_suites,num_vagas_garagem,num_andares,tipo_imovel,estado_construcao,fonte,imovel_lancamento,bl_temporada,vista_mar_bin,mobiliado_bin,preco_m2
0,91.0,NaN,NaN,3,1,1,2,0,Apartamento,desconhecido,Viva Real,0,0,0,0,12751.219780
1,46.0,600.0,700.0,1,2,1,2,7,Apartamento,desconhecido,Viva Real,0,0,0,0,10000.000000
2,75.0,0.0,600.0,3,2,1,2,0,Apartamento,desconhecido,Viva Real,0,0,0,0,4133.333333


{'raw_url': 'https://raw.githubusercontent.com/zilioalberto/N3_Ciencia_Dados/main/data/dataset_original/tb_mercadoimob.csv',
 'raw_path': '/content/data/dataset_original/tb_mercadoimob.csv',
 'out_path': '/content/data/dataset_processado_N3/base_modelagem.csv',
 'raw_shape': [9236, 41],
 'after_basic_filters_shape': [7780, 44],
 'model_shape': [7780, 16],
 'features': ['area',
  'iptu',
  'taxa_condominial',
  'num_quartos',
  'num_banheiros',
  'num_suites',
  'num_vagas_garagem',
  'num_andares',
  'tipo_imovel',
  'estado_construcao',
  'fonte',
  'imovel_lancamento',
  'bl_temporada',
  'vista_mar_bin',
  'mobiliado_bin'],
 'target': 'preco_m2',
 'notes': ['O dataset foi filtrado para tipo_negocio==Venda (se a coluna existir).',
  'preco_m2 possui clipping por IQR para reduzir influência de outliers sem descartar linhas.',
  'O CSV é baixado do GitHub e cacheado em RAW_PATH quando não existir localmente.']}

In [23]:
# 3) Definir target e features (prioriza etl_report.json)
target = report.get("target", "preco_m2")
if target not in df.columns:
    raise ValueError(f"Target '{target}' não encontrada no CSV. Colunas disponíveis: {df.columns.tolist()}")

features = report.get("features")
if not features:
    features = [c for c in df.columns if c != target]

X = df[features].copy()
y = df[target].copy()

print("Target:", target)
print("Nº features:", len(features))
X.dtypes.value_counts()


Target: preco_m2
Nº features: 15


,count
int64,9
float64,3
object,3


In [24]:
# 4) Separar colunas numéricas e categóricas
numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
categorical_features = [c for c in X.columns if c not in numeric_features]

print("Numéricas:", numeric_features)
print("Categóricas:", categorical_features)


Numéricas: ['area', 'iptu', 'taxa_condominial', 'num_quartos', 'num_banheiros', 'num_suites', 'num_vagas_garagem', 'num_andares', 'imovel_lancamento', 'bl_temporada', 'vista_mar_bin', 'mobiliado_bin']
Categóricas: ['tipo_imovel', 'estado_construcao', 'fonte']


In [25]:
# 5) Split treino/teste
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Train:", X_train.shape, y_train.shape)
print("Test :", X_test.shape, y_test.shape)


Train: (6224, 15) (6224,)
Test : (1556, 15) (1556,)


In [26]:
# 6) Pré-processamento (pipeline)
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ],
    remainder="drop",
)

preprocess


ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['area', 'iptu', 'taxa_condominial',
                                  'num_quartos', 'num_banheiros', 'num_suites',
                                  'num_vagas_garagem', 'num_andares',
                                  'imovel_lancamento', 'bl_temporada',
                                  'vista_mar_bin', 'mobiliado_bin']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['tipo_imovel', 'estado_construcao',
                                  'fonte'])])

In [27]:
# 7) Modelos (mínimo 3)
models = {
    "LinearRegression": LinearRegression(),
    "Ridge(alpha=1.0)": Ridge(alpha=1.0, random_state=42),
    "RandomForest(n=300)": RandomForestRegressor(
        n_estimators=300,
        random_state=42,
        n_jobs=-1
    ),
}
models


{'LinearRegression': LinearRegression(),
 'Ridge(alpha=1.0)': Ridge(random_state=42),
 'RandomForest(n=300)': RandomForestRegressor(n_estimators=300, n_jobs=-1, random_state=42)}

In [28]:
# 8) Treinar + avaliar (3 métricas: MAE, RMSE, R²) + tabela comparativa
def evaluate(y_true, y_pred) -> dict:
    mae = mean_absolute_error(y_true, y_pred)
    mse = mean_squared_error(y_true, y_pred)   # sem squared
    rmse = float(np.sqrt(mse))
    r2 = r2_score(y_true, y_pred)
    return {"MAE": mae, "RMSE": rmse, "R2": r2}


rows = []
pipelines = {}

for name, model in models.items():
    pipe = Pipeline(steps=[
        ("preprocess", preprocess),
        ("model", model),
    ])

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)

    m = evaluate(y_test, y_pred)
    rows.append({"Modelo": name, **m})
    pipelines[name] = pipe

results = pd.DataFrame(rows).sort_values("RMSE").reset_index(drop=True)
results


,Modelo,MAE,RMSE,R2
0,RandomForest(n=300),875.451425,1783.560310,0.833903
1,Ridge(alpha=1.0),2866.264349,3704.410982,0.283486
2,LinearRegression,2865.953088,3704.543449,0.283434


In [29]:
# 9) Salvar tabela comparativa local (para commitar no Git depois)
OUT_RESULTS = OUT_DIR / "comparacao_modelos.csv"
results.to_csv(OUT_RESULTS, index=False)
print("Tabela salva em:", OUT_RESULTS)

results


Tabela salva em: /content/data/dataset_processado_N3/comparacao_modelos.csv


,Modelo,MAE,RMSE,R2
0,RandomForest(n=300),875.451425,1783.560310,0.833903
1,Ridge(alpha=1.0),2866.264349,3704.410982,0.283486
2,LinearRegression,2865.953088,3704.543449,0.283434


In [30]:
# 10) Escolher melhor modelo (menor RMSE) e salvar como modelo_final.pkl (com backup automático)
best_name = results.loc[0, "Modelo"]
best_pipe = pipelines[best_name]

print("Melhor modelo por RMSE:", best_name)

MODEL_PATH = ROOT / "modelo_final.pkl"
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

if MODEL_PATH.exists():
    backup = ROOT / f"modelo_final_backup_{timestamp}.pkl"
    MODEL_PATH.replace(backup)
    print("Backup criado:", backup)

joblib.dump(best_pipe, MODEL_PATH)
print("Modelo salvo em:", MODEL_PATH)


Melhor modelo por RMSE: RandomForest(n=300)
Backup criado: /content/modelo_final_backup_20251130_210910.pkl
Modelo salvo em: /content/modelo_final.pkl


In [31]:
# 11) Demonstração rápida (Deploy): carregar e prever um dado "novo"
loaded = joblib.load(MODEL_PATH)

# Exemplo: usar uma linha do conjunto de teste como entrada
example = X_test.iloc[[0]].copy()
pred = float(loaded.predict(example)[0])

print("Entrada (1 linha):")
display(example)
print(f"Previsão de {target}: {pred:.2f}")


Entrada (1 linha):


,area,iptu,taxa_condominial,num_quartos,num_banheiros,num_suites,num_vagas_garagem,num_andares,tipo_imovel,estado_construcao,fonte,imovel_lancamento,bl_temporada,vista_mar_bin,mobiliado_bin
7676,140.0,3428.0,1340.0,3,2,2,2,0,Apartamento,desconhecido,Viva Real,0,0,0,0


Previsão de preco_m2: 12246.93


## Observações
- Este notebook **lê os dados diretamente do GitHub** (via `raw.githubusercontent.com`).
- As saídas `comparacao_modelos.csv` e `modelo_final.pkl` são geradas **localmente** (na sua máquina) e devem ser **commitadas** no repositório para compor a entrega.
